In [1]:
import os
import torch

from pyro.distributions import Uniform

from pyciemss.ODE.base import PetriNetODESystem, GaussianNoisePetriNetODESystem
from pyciemss.ODE.events import Event, ObservationEvent, LoggingEvent, StartEvent, DynamicStopEvent
import pyciemss

In [2]:
STARTERKIT_PATH = "test/models/starter_kit_examples/"
MIRA_PATH = "test/models/evaluation_examples/"

filename = "CHIME-SIR/model_petri.json"
filename = os.path.join(STARTERKIT_PATH, filename)

model = GaussianNoisePetriNetODESystem.from_mira(filename)
model.load_start_event(0.0, {"S": 0.9, "I": 0.1, "R": 0.0})
model.load_logging_events(torch.linspace(1, 10, 10))

solution = model()

# See that the solution returns a dictionary where each value has length 10, one for each logging event.
assert len(solution["I"]) == len(solution["R"]) == len(solution["S"]) == 10

solution

{'I': tensor([0.0883, 0.0734, 0.0582, 0.0445, 0.0332, 0.0243, 0.0175, 0.0125, 0.0089,
         0.0063]),
 'R': tensor([0.0864, 0.1604, 0.2204, 0.2671, 0.3024, 0.3284, 0.3474, 0.3610, 0.3706,
         0.3775]),
 'S': tensor([0.8253, 0.7662, 0.7214, 0.6884, 0.6644, 0.6473, 0.6351, 0.6265, 0.6205,
         0.6162])}

In [3]:
# Remove the logging events.
model.delete_logging_events()

# Add observation events.
# TODO: implement a pyciemss.condition operation that returns a model with observation_events
model._observation_events = [ObservationEvent(1.3, "R", torch.tensor(1.0)), ObservationEvent(2.3, "I", torch.tensor(0.2))]
model._construct_static_events()

model._static_events

model()


{'I': tensor([]), 'R': tensor([]), 'S': tensor([])}

In [4]:
# Show that inference works.

from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 7.7232
[iteration 0026] loss: 5.1529
[iteration 0051] loss: 3.8828
[iteration 0076] loss: 3.2353


In [8]:
# use pyro predictive
from pyro.infer import Predictive

# Remove the observation events and add logging events.

model.delete_observation_events()
model.load_logging_events(torch.linspace(1, 10, 10))


# Get the return value of model.
predictions = Predictive(model, guide=guide, num_samples=10)()

In [9]:
predictions

{'a_beta': tensor([0.9714, 0.9642, 0.9368, 0.9756, 1.0346, 0.9873, 0.9798, 0.9226, 1.0033,
         1.0057]),
 'a_gamma': tensor([1.0449, 0.9688, 0.9644, 0.9423, 0.9799, 0.9895, 1.0262, 0.9967, 0.9844,
         0.9640]),
 'I_sol': tensor([[0.0811, 0.0617, 0.0449, 0.0317, 0.0219, 0.0149, 0.0100, 0.0067, 0.0045,
          0.0030],
         [0.0869, 0.0707, 0.0547, 0.0408, 0.0296, 0.0211, 0.0149, 0.0104, 0.0072,
          0.0050],
         [0.0854, 0.0685, 0.0525, 0.0389, 0.0281, 0.0200, 0.0141, 0.0098, 0.0068,
          0.0047],
         [0.0901, 0.0756, 0.0601, 0.0459, 0.0340, 0.0247, 0.0176, 0.0125, 0.0088,
          0.0061],
         [0.0910, 0.0765, 0.0605, 0.0457, 0.0334, 0.0239, 0.0168, 0.0117, 0.0081,
          0.0056],
         [0.0868, 0.0703, 0.0540, 0.0399, 0.0288, 0.0203, 0.0142, 0.0098, 0.0068,
          0.0046],
         [0.0832, 0.0648, 0.0481, 0.0345, 0.0242, 0.0167, 0.0114, 0.0077, 0.0052,
          0.0035],
         [0.0818, 0.0631, 0.0467, 0.0335, 0.0236, 0.0164, 0.011

In [ ]:
# SCAFFOLDING FOR DYNAMIC EVENT HANDLING BELOW

import pyro
from torch import Tensor
from torchdiffeq import odeint_event, odeint

Time = Tensor
State = tuple[Tensor, ...]

class BaseODEModel(pyro.nn.PyroModule):

    def __init__(self, static_events: list[Event]):
        super().__init__()
        # This is a list of events that are always sorted by time.
        # TODO: probably pre-sort this list just in case.
        self.static_events = static_events

    def deriv(self, t: Time, state: State) -> State:
        raise NotImplementedError

    @pyro.nn.pyro_method
    def param_prior(self):
        raise NotImplementedError

    @pyro.nn.pyro_method
    def initial_conditions_prior(self):
        raise NotImplementedError

    # @pyro.nn.pyro_method
    # def observation_model(self, state: State, tspan: Time, ?) -> ?: …

    def solve(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        
        current_state = initial_state
        current_time = initial_time

        solution = torch.tensor([])

        for i, static_event in enumerate(self.static_events):
            
            # TODO: change below
            # Note: Immediate goal is to get self.solve() to generate ode solutions that stop at the event times.
            # Note: Next we will add log likelihood at those events using an observation model.

            # Note: each static_event is an Event object, which has a forward() method that returns a Tensor.
            # TODO: Chad, could you please make this actually make sense.
            event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)
    
            # TODO: Add log likelihood for event_solution.
            # Note: This will be done by adding an observation model to the ODE model and calling it with the event_solution as an argument.

            solution = torch.cat([solution, event_solution], dim=-1)
            # current = event_solution[-1]


    def forward(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        # Sample parameters from the prior. These parameters are generated as attributes of the model.
        self.param_prior()

        # TODO: Sample initial conditions from the prior instead of taking them as input.

        # Solve the ODE, taking into account any intervention and conditioning events.
        return self.solve(initial_state, initial_time)




In [ ]:
event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)

In [ ]:

# TODO: Chad, initialize a stop event here.
# stop_event = ...

# TODO: Chad, create a model the subclasses BaseODEModel and has all of the above missing methods implemented.
# deriv, param_prior, initial_conditions_prior, observation_model
# class ActualODEModel(BaseODEModel):
#   ...

# TODO: Chad, instantiate the model here.
# model = ActualODEModel([stop_event])

# TODO: Chad, Call model.forward() here.
# solution, _ = model(initial_state, tspan)

In [7]:
observations = [ObservationEvent(torch.tensor([2.1]), {"I": torch.tensor([0.1])})]


In [8]:
conditioned_model = pyciemss.condition(model, observations)

AttributeError: module 'pyciemss' has no attribute 'condition'